In [1]:
# -*- coding: utf-8 -*-

# Download Files CSV to use - https://github.com/wcota/covid19br - Great Work


In [2]:
%%!
pip install --trusted-host pypi.org ipython-sql
%%!
pip install wget
%%!
pip install psycopg2

['Requirement already satisfied: ipython-sql in c:\\osgeo4~1\\apps\\python37\\lib\\site-packages (0.3.9)',
 'Requirement already satisfied: prettytable in c:\\osgeo4~1\\apps\\python37\\lib\\site-packages (from ipython-sql) (0.7.2)',
 'Requirement already satisfied: ipython-genutils>=0.1.0 in c:\\osgeo4~1\\apps\\python37\\lib\\site-packages (from ipython-sql) (0.2.0)',
 'Requirement already satisfied: sqlparse in c:\\osgeo4~1\\apps\\python37\\lib\\site-packages (from ipython-sql) (0.3.1)',
 'Requirement already satisfied: sqlalchemy>=0.6.7 in c:\\osgeo4~1\\apps\\python37\\lib\\site-packages (from ipython-sql) (1.3.16)',
 'Requirement already satisfied: six in c:\\users\\andre\\appdata\\roaming\\python\\python37\\site-packages (from ipython-sql) (1.12.0)',
 'Requirement already satisfied: ipython>=1.0 in c:\\osgeo4~1\\apps\\python37\\lib\\site-packages (from ipython-sql) (7.13.0)',
 'Requirement already satisfied: pygments in c:\\osgeo4~1\\apps\\python37\\lib\\site-packages (from ipython

In [3]:
#Just to first time run
lastd=9
lastm=5
lasty=2020

In [4]:
import wget
import os
from datetime import datetime

url = "https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-total.csv"
url2 = "https://github.com/wcota/covid19br/raw/master/cases-brazil-cities.csv"
filePath='C:/OSGeo4W64/bin/Andre/Covid-19Brazil/Tables/cases-brazil-total.csv'
filePath2='C:/OSGeo4W64/bin/Andre/Covid-19Brazil/Tables/cases-brazil-cities.csv'

url3 = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv"

filePath3='C:/OSGeo4W64/bin/Andre/Covid-19Brazil/Tables/cases_country.csv'

url4="https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv&filename=time_series_covid19_confirmed_global.csv"
filePath4='C:/OSGeo4W64/bin/Andre/Covid-19Brazil/Tables/time_total_world.csv'
now = datetime.now()
d= int(now.strftime("%d"))
m=int(now.strftime("%m"))
y= int(now.strftime("%Y"))

if lastd<d:
    print(lastd)
    print(d)
    #check if the month or year has changed
    lastd=d
    if lastm<m:
        lastd=1
    if lasty<y:
        lastd=1
        lastm=1
    #---------------------------------
    
    if os.path.exists(filePath):
        os.remove(filePath)
    else:
        print("Can not delete the file as it doesn't exists")

    if os.path.exists(filePath2):
        os.remove(filePath2)
        
    if os.path.exists(filePath3):
        os.remove(filePath3)
    else:
        print("Can not delete the file as it doesn't exists")
    try:
        wget.download(url,filePath )
        wget.download(url2, filePath2)
        wget.download(url3,filePath3 )
        wget.download(url4,filePath4 )

        print("\nSuccessfully downloaded")
    except:
        print("Download Problems")
else:
    print("Files already recent")
#url = "ftp://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc250/versao2019/lista_de_nomes_geograficos/bc250_nomesgeograficos_2019-10-29.csv"
#wget.download(url, 'C:/OSGeo4W64/bin/Andre/Covid-19Brazil/Tables/bc250_nomesgeograficos_2019-10-29.csv')

#url = "ftp://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc250/versao2019/shapefile/bc250_shapefile_06_11_2019.zip"
#wget.download(url, 'C:/OSGeo4W64/bin/Andre/Covid-19Brazil/Tables/bc250_shapefile_06_11_2019.zip')


9
13
-1 / unknown........................................................................] 20389 / 20389
Successfully downloaded


# Loading CSV files to a PostGreSQL database

In [5]:
import psycopg2
import csv
conn = psycopg2.connect("host=localhost dbname=Miebiom user=postgres password=1234")
cur = conn.cursor()

In [6]:

cur.execute(""" DROP TABLE IF EXISTS brasiltotal; """)
conn.commit()


In [7]:

cur.execute("""
    CREATE TABLE brasiltotal(
    country text,
    state text,
    totalCases text,
    totalCasesMS text,
    notConfirmedByMS text,
    deaths text,
    deathsMS text,
    URL text,
    deaths_per_100k_inhabitants text,
    totalCases_per_100k_inhabitants text,
    deaths_by_totalCases text,
    recovered text,
    suspects text,
    tests text,
    tests_per_100k_inhabitants text
)
""")


In [8]:



with open('C:/OSGeo4W64/bin/Andre/Covid-19Brazil/Tables/cases-brazil-total.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # Skip the header row.
    for row in reader:
        cur.execute(
        "INSERT INTO brasiltotal VALUES ( %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s)",
        row
    )

    

In [9]:
conn.commit()

In [10]:
import psycopg2
conn = psycopg2.connect("host=localhost dbname=Miebiom user=postgres password=1234")
cur = conn.cursor()
cur.execute(""" DROP TABLE IF EXISTS brasilcasescities; """)
conn.commit()

In [11]:


cur.execute("""
    CREATE TABLE brasilcasescities(
    country text, 
    state text,
    city text,
    ibgeID text,
    deaths text,
    totalCases text,
    deaths_per_100k_inhabitants text,
    totalCases_per_100k_inhabitants text,
    deaths_by_totalCases text
)
""")

with open('C:/OSGeo4W64/bin/Andre/Covid-19Brazil/Tables/cases-brazil-cities.csv', 'rt', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader) # Skip the header row.
    for row in reader:
        cur.execute(
        "INSERT INTO brasilcasescities VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s)",
        row
    )

    

In [12]:
conn.commit()

In [21]:
import psycopg2
import csv

conn = psycopg2.connect("host=localhost dbname=Miebiom user=postgres password=1234")
cur = conn.cursor()
cur.execute(""" DROP TABLE IF EXISTS worldcases; """)
conn.commit()

In [22]:
cur.execute("""
    CREATE TABLE worldcases(
    Country_Region text, 
    Last_Update text,
    Lat text,
    Long_ text,
    Confirmed INTEGER ,
    Deaths INTEGER ,
    Recovered INTEGER ,
    Active INTEGER 
    
    
)
""")



In [30]:
with open('C:/OSGeo4W64/bin/Andre/Covid-19Brazil/Tables/cases_country.csv', 'rt', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader) # Skip the header row.
    for row in reader:
       
        print(row[0], row[1], row[2], row[3], row[4], row[5], row[6],row[7])

        cur.execute(
        "INSERT INTO worldcases VALUES ( %s, %s, %s, %s, %s, %s, %s, %s)",
        (row[0], row[1], row[2], row[3], row[4], row[5], row[6],row[7]))
        
     
    

Australia 2020-05-13 05:32:26 -25.0 133.0 6980 98 6271 611
Austria 2020-05-13 05:32:26 47.5162 14.5501 15961 623 14148 1190
Canada 2020-05-13 05:32:26 60.001000000000005 -95.001 72419 5300 34055 33064
China 2020-05-13 05:32:26 30.5928 114.3055 84018 4637 79224 157
Denmark 2020-05-13 05:32:26 56.0 10.0 10789 527 8778 1484
Finland 2020-05-13 05:32:26 61.9241 25.7482 6003 275 4300 1428
France 2020-05-13 05:32:26 46.2276 2.2137 178349 26994 57898 93457
Germany 2020-05-13 05:32:26 51.1657 10.4515 173171 7738 147200 18233
Iceland 2020-05-13 05:32:26 64.9631 -19.0208 1801 10 1776 15
Ireland 2020-05-13 05:32:26 53.1424 -7.6921 23242 1488 17110 4644
Italy 2020-05-13 05:32:26 41.8719 12.5674 221216 30911 109039 81266
Netherlands 2020-05-13 05:32:26 52.3167 5.55 43183 5529 149 37505
Norway 2020-05-13 05:32:26 60.472 8.4689 8157 228 32 7897
Russia 2020-05-13 05:32:26 61.523999999999994 105.3188 232243 2116 43512 186615
Sweden 2020-05-13 05:32:26 60.1282 18.6435 27272 3313 4971 18988
Switzerland 20

In [31]:
conn.commit()